# In Class Notebook, Week 14

You can always paste the URL of this notebook (https://github.com/UIUC-iSchool-DataViz/is445_spring2021/blob/master/week14/inClass_week14.ipynb ) into the nbviewer interface for a plain-text rendering:

https://kokes.github.io/nbviewer.js/viewer.html

Let me write something!

Before yt, let's check out ipyvolume

## ipyvolume

In [1]:
# import our usual things
import numpy as np
import pandas as pd
import ipyvolume

In [2]:
x, y, z = np.random.random((3, 100)) # 3 dimensions, 100 points total

In [7]:
ipyvolume.quickscatter?

In [8]:
# quick 3d scatter plot
ipyvolume.quickscatter(x,y,z, size=5, marker='diamond')